In [2]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import pandas as pd
import json
from langchain_community.embeddings import HuggingFaceEmbeddings

In [3]:
# Configuración del modelo
MODEL = "qwen2.5:7b"
llm = Ollama(
    model=MODEL,
    temperature=0.9,
    top_p=0.9,
    num_ctx=4096,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
# Ejemplo de generación inicial de datos
#response = llm.invoke("genera una tabla con dos filas, valores variados y realistas y los siguientes campos: device_id, timestamp, bandwidth_mbps, latency_ms, packet_loss, signal_strength_dbm, cell_id, connection_type. Responde SOLO con el JSON puro, sin markdown, sin comillas triples, sin explicaciones.")
#display(response)   

C:\Users\luis.sanmartin\AppData\Local\Temp\ipykernel_6628\1485541037.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
C:\Users\luis.sanmartin\AppData\Local\Temp\ipykernel_6628\1485541037.py:3: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


In [4]:
# Cargar archivo JSON localmente (esto puede ser un archivo de ejemplo)
json_file = "json_dataset.json"  # Asegúrate de que este archivo esté en la misma carpeta o proporciona la ruta completa

# Leer el archivo JSON de forma asíncrona
with open(json_file, "r") as f:
    contents = f.read()
    json_data = json.loads(contents)

# Extraer registros según la estructura del JSON
if isinstance(json_data, dict) and "data" in json_data:
    # Si es la estructura de MongoDB
    records = json_data["data"]
elif isinstance(json_data, list):
    # Si es una lista directa de registros
    records = json_data
else:
    # Si es un único registro
    records = [json_data]

# Convertir los registros a DataFrame y limpiar
df = pd.DataFrame(records)

# Eliminar columnas no deseadas si existen
columns_to_drop = ['_id', 'dataset_id']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# Verificar columnas resultantes
fields = df.columns.tolist()
print(fields)

# Crear documentos para embeddings (concatenar columnas en una cadena)
documents = []
for _, row in df.iterrows():
    doc_text = " ".join([f"{col}: {val}" for col, val in row.items()])
    documents.append(doc_text)

# Crear un splitter para dividir los textos largos en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Chunks más pequeños
    chunk_overlap=50       # Menor superposición
)

# Dividir los documentos en fragmentos
split_docs = []
for doc in documents:
    split_docs.extend(text_splitter.split_text(doc))

# Definir el modelo de embeddings de HuggingFace
EMBEDDINGS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL)

# Crear y guardar los embeddings en ChromaDB
db = Chroma.from_texts(texts=split_docs, embedding=embeddings)

# Verifica el número de fragmentos generados
print(f"Se generaron {len(split_docs)} fragmentos de texto.")
df.head()

['device_id', 'timestamp', 'bandwidth_mbps', 'latency_ms', 'packet_loss', 'signal_strength_dbm', 'cell_id', 'connection_type']


C:\Users\luis.sanmartin\AppData\Local\Temp\ipykernel_6628\717872230.py:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL)
c:\Users\luis.sanmartin\AppData\Local\miniconda3\envs\genia_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Se generaron 1000 fragmentos de texto.


,device_id,timestamp,bandwidth_mbps,latency_ms,packet_loss,signal_strength_dbm,cell_id,connection_type
0,5851,2024-12-13T14:46:24.173261,536.156239,6.551220,0.037045,-101.327696,57,MIMO
1,4856,2024-12-13T15:46:24.173271,485.536208,9.675234,0.040759,-99.914445,66,MIMO
2,3914,2024-12-13T16:46:24.173273,420.810843,8.836038,0.049106,-98.675641,59,MIMO
3,9267,2024-12-13T17:46:24.173274,736.451439,8.656040,0.056578,-80.390779,68,Carrier Aggregation
4,4908,2024-12-13T18:46:24.173275,524.275720,11.614536,0.022364,-109.168636,98,MIMO


In [ ]:
retriever = db.as_retriever()
# Definir un prompt base
prompt_template = PromptTemplate(
    template="""
    eres un experto en ciberseguridad y basándote en este dataset y sus campos:
    {context}

    Responde a la pregunta del usuario:
    {question}
    """,
    input_variables=["context", "question"]
)

# Configurar la cadena RAG
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt_template}
)

# Construir la pregunta dinámica con los campos
fields_str = ', '.join(fields)  # Convertir la lista de columnas a una cadena
num_samples = 3  # Número de registros a generar

# Ejemplo de consulta al sistema
question = f"Genera una tabla con {num_samples} filas, valores random y los siguientes campos: {fields_str}. Responde SOLO con el JSON puro, sin markdown, sin comillas triples, sin explicaciones."
response = rag_chain.invoke(question)
display(response)


In [ ]:

# Extraer y limpiar el campo 'result'
raw_result = response['result']

# 1. Separar los objetos JSON por línea y limpiar el formato
json_objects = raw_result.split("}\n{")
json_objects = [obj if obj.startswith("{") else "{" + obj for obj in json_objects]
json_objects = [obj if obj.endswith("}") else obj + "}" for obj in json_objects]

# 2. Convertir cada objeto en un diccionario
data = [json.loads(obj) for obj in json_objects]

# 3. Crear un DataFrame a partir de los datos
df_generated = pd.DataFrame(data)

# 4. Guardar el DataFrame como archivo CSV
output_csv = "output.csv"
df_generated.to_csv(output_csv, index=False)


df_generated.head()